## Data

In [3]:
file = open("characterGenData.txt",encoding = "utf-8").read()

In [4]:
import numpy
import sys
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

C:\Users\newgh\Anaconda3\lib\site-packages\sklearn\feature_extraction\image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):
C:\Users\newgh\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.o

In [5]:
def tokenize_words(input):
    input = input.lower()

    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(input)

    filtered = filter(lambda token: token not in stopwords.words('english'), tokens)
    return " ".join(filtered)

In [6]:
processed_inputs = tokenize_words(file)

In [7]:
chars = sorted(list(set(processed_inputs)))
char_to_num = dict((c, i) for i, c in enumerate(chars))

In [8]:
input_len = len(processed_inputs)
vocab_len = len(chars)
print ("Total number of characters:", input_len)
print ("Total vocab:", vocab_len)

Total number of characters: 107450
Total vocab: 42


In [9]:
seq_length = 100
x_data = []
y_data = []

input will be a sequence of 100 characters and output with be 1 char that comes after those 100

In [11]:
for i in range(0, input_len - seq_length, 1):
    in_seq = processed_inputs[i:i + seq_length]
    out_seq = processed_inputs[i + seq_length]
    x_data.append([char_to_num[char] for char in in_seq])
    y_data.append(char_to_num[out_seq])

In [12]:
n_patterns = len(x_data)

reshape + chars are represented as 0-->47 we transform them to 0 --> 47/48 so they can be interpreted by the sigmoid func

In [13]:
X = numpy.reshape(x_data, (n_patterns, seq_length, 1))
X = X/float(vocab_len)

In [14]:
y = np_utils.to_categorical(y_data)
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [15]:
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))

In [16]:
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [17]:
filepath = "model_weights_saved.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
desired_callbacks = [checkpoint]

In [25]:
model.fit(X, y, epochs=25, batch_size=256, callbacks=desired_callbacks)

Epoch 1/25
420/420 [==============================] - ETA: 0s - loss: 2.5494
Epoch 00001: loss improved from 2.63295 to 2.54943, saving model to model_weights_saved.hdf5
420/420 [==============================] - 96s 219ms/step - loss: 2.5494
Epoch 2/25
420/420 [==============================] - ETA: 0s - loss: 2.4849
Epoch 00002: loss improved from 2.54943 to 2.48489, saving model to model_weights_saved.hdf5
420/420 [==============================] - 92s 219ms/step - loss: 2.4849
Epoch 3/25
420/420 [==============================] - ETA: 0s - loss: 2.4208
Epoch 00003: loss improved from 2.48489 to 2.42076, saving model to model_weights_saved.hdf5
420/420 [==============================] - 91s 216ms/step - loss: 2.4208
Epoch 4/25
420/420 [==============================] - ETA: 0s - loss: 2.3637
Epoch 00004: loss improved from 2.42076 to 2.36371, saving model to model_weights_saved.hdf5
420/420 [==============================] - 91s 216ms/step - loss: 2.3637
Epoch 5/25
420/420 [========

In [26]:
model.load_weights(filepath)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [27]:
num_to_char = dict((i, c) for i, c in enumerate(chars))

picking a random pattern from the training data

In [28]:
start = numpy.random.randint(0, n_patterns - 1)
pattern = x_data[start]
print("Random Seed:")
print("\"", ''.join([num_to_char[value] for value in pattern]), "\"")

Random Seed:
" d found father inexorable quitted country returned heard former mistress married according inclinati "


In [29]:
for i in range(200):
    #prepare the input
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(vocab_len)
    
    #pridict the output letter
    prediction = model.predict(x, verbose=0)
    #pick the most probable letter
    index = numpy.argmax(prediction)
    #get the char and print it
    result = num_to_char[index]
    sys.stdout.write(result)
    
    #append the index and move the pattern to the next squence
    pattern.append(index)
    pattern = pattern[1:len(pattern)]

on sear sense sear sense sear sense sear sense sear sense sear sense sear sense sear sense sear sense sear sense sear sense sear sense sear sense sear sense sear sense sear sense sear sense sear sense